In [ ]:
# default_exp exp.desc.metrics.java

# Exploration of your data

> This module comprises some of the statistical and inference techniques to describe the inner properties of software data. The submodules might include:
>
> - Descriptive statistics
> - Software Metrics
> - Information Theory
> - Learning Principels Detection (Occams' Razor, Biased data, and Data Snooping)
> - Inference: Probabilistic and Causal

### Specifically in this module

> - Cyclomatic complexity (CYCLO)
> - Number of lines of code (NLOC)
> - Lack of Cohesion of Methods 5 (LCOM5)

### Current limitations

> - Can only compute LCOM5 for Java files
> - Can theoretically compute Cyclomatic Complexity for 
> > - C
> > - C++ (works with C++14)
> > - Java
> > - C# (C Sharp)
> > - JavaScript (With ES6 and JSX)
> > - Objective-C
> > - Swift
> > - Python
> > - Ruby
> > - TTCN-3
> > - PHP
> > - Scala
> > - GDScript
> > - Golang
> > - Lua
> > - Rust

In [ ]:
# #hide
# from nbdev.showdoc import *

In [ ]:
# export
# Imports
import pandas as pd
from numpy import mean, std
from statistics import median
from scipy.stats import sem, t
import lizard
import matplotlib.pyplot as plt
from tree_sitter import Language, Parser, Node
#Decoding files
import chardet
from bs4 import UnicodeDammit


# TODO: Remove when mongo call is implemented
import os

In [ ]:
!git clone https://github.com/tree-sitter/tree-sitter-java

Cloning into 'tree-sitter-java'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1510 (delta 115), reused 172 (delta 72), pack-reused 1285
Receiving objects: 100% (1510/1510), 13.05 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (935/935), done.


# Get_unicode & test

In [ ]:
# export
'''
Detects file encoding and returns unicode.
Inspired by http://reinvantveer.github.io/2017/05/19/unicode-dammit.html

:param file_path: file path of file
:returns: unicode string of the file
:raises ValueError: empty or invalud csv file
'''
def get_unicode(file_path):
    with open(file_path, 'rb') as f:
        detection = chardet.detect(f.read())
        
    enc = detection["encoding"]
    if detection["encoding"] == "ascii":
        with open(file_path, encoding="ascii") as f:
            data = f.read()
    elif detection["encoding"] == "ISO-8859-9":
        with open(file_path, encoding="utf-8") as f:
            enc = "utf-8"
            data = f.read()
    else:
        try:
            # Try to open as non unicode file
            with open(file_path, encoding=detection["encoding"]) as f:
                data = f.read()
        except Exception as e:
            raise ValueError(f"Cannot return dictionary from empty or invalid csv file {file_path} due to {e}")

    if not data:
        raise ValueError(f"Cannot return dictionary from empty or invalid csv file {file_path}")

    return UnicodeDammit(data).unicode_markup, enc

In [ ]:
# Setting up test data
file_path = "test_utf_8_file.txt"
with open(file_path, 'w', encoding="utf-8") as f:
    f.write("Hello, this is a test file")
# Calling function
text, encoding = get_unicode(file_path)
# Cleanup
os.remove(file_path)
# Displaying output
print(encoding)
print(text)

NameError: name 'get_unicode' is not defined

In [ ]:
# export
'''
Loads files from a specified folder into a pandas dataframe

:param folder_path: path to folder
:returns: corpus_data dataframe
'''
def simulate_getting_dataframes_from_mongo(folder_path):
    corpus_data = {"system": [], "name": [], "ground_truth": [], "contents": [], "encoding": []}
    for file in os.listdir(folder_path):
        if not os.path.isdir(os.path.join(folder_path, file)) and file != ".DS_Store":
            corpus_data["system"].append(None)
            corpus_data["name"].append(file)
            corpus_data["ground_truth"].append("src")
            contents, enc = get_unicode(os.path.join(folder_path, file))
            corpus_data['encoding'].append(enc)
            corpus_data['contents'].append(contents)
    corpus_df = pd.DataFrame(data = corpus_data)
    return corpus_df

# Add_mccabe_metrics & test

In [ ]:
# export
'''
Adds information about function length and cyclomatic complexity for classes to a dataframe

:param df: dataframe 
:param data_col: data
:param name_col: name
:returns: dataframe with added columns
'''
def add_mccabe_metrics(df, data_col, name_col):
    num_funcs = []
    class_ccn = []
    avg_func_ccn = []
    avg_func_nloc = []
    for i in range(len(df)):
        file_num_funcs = []
        file_class_ccn = []
        file_avg_func_ccn = []
        file_avg_func_nloc = []
        metrics = lizard.analyze_file.analyze_source_code(df[name_col][i], df[data_col][i])
        class_dict = {}
        for func in metrics.function_list:
            class_name = '::'.join(func.name.split("::")[:-1])
            if class_name in class_dict:
                class_dict[class_name].append(func)
            else:
                class_dict[class_name] = [func]
        for class_key in class_dict:
            total_class_ccn = 0
            total_class_nloc = 0
            for func in class_dict[class_key]:
                total_class_ccn += func.cyclomatic_complexity
                total_class_nloc += func.length
            file_num_funcs.append(len(class_dict[class_key]))
            file_class_ccn.append(total_class_ccn)
            file_avg_func_ccn.append(total_class_ccn/len(class_dict[class_key]))
            file_avg_func_nloc.append(total_class_nloc/len(class_dict[class_key]))

        num_funcs.append(file_num_funcs)
        class_ccn.append(file_class_ccn)
        avg_func_ccn.append(file_avg_func_ccn)
        avg_func_nloc.append(file_avg_func_nloc)

    df["num_funcs"] = num_funcs
    df["class_ccn"] = class_ccn
    df["avg_func_ccn"] = avg_func_ccn
    df["avg_func_nloc"] = avg_func_nloc
    return df

In [ ]:
# Setting up test data
file_path = "test_java_file.java"
folder_path = "test_dir"
os.mkdir(folder_path)
with open(os.path.join(folder_path, file_path), 'w') as f:
    f.write("public int main() {}")
# Calling function
pd_dataframe = simulate_getting_dataframes_from_mongo(folder_path)
pd_dataframe = add_mccabe_metrics(pd_dataframe, "contents", "name")
# Cleanup
os.remove(os.path.join(folder_path,file_path))
os.rmdir(folder_path)
# Displaying output
pd.set_option('display.expand_frame_repr', False)
pd_dataframe.head()

In [ ]:
#export
'''
Creates a dictionary of tree-sitter parsers for select languages

:param path: None
:returns: dictionary with java and java language
'''
def create_parser_builds(path=None):
    Language.build_library(
        # Store the library in the `build` directory
        'build/my-languages.so',

        # Include one or more languages
        [
            'tree-sitter-java'
        ]
    )

    JAVA_LANGUAGE = Language('build/my-languages.so', 'java')
    
    return {"java":JAVA_LANGUAGE}

In [ ]:
#export
'''
Returns language specific keywords for a parser to find
'''
def lang_keywords():
    keyword_dict = {}
    keyword_dict["java"] = {"class": "class_declaration", "method":"method_declaration", "field_dec":"field_declaration", "field_name":"identifier"}
    return keyword_dict

In [ ]:
#export
'''
Recursively searches an AST for class nodes

:param root_node: root node to begin search
:param key: key to search
:returns: list of class nodes
'''
def find_class_nodes(root_node, key):
    node_list = []
    def rec_class_search(node):
        if node.type == key["class"]:
            node_list.append(node)
        for child in node.children:
            rec_class_search(child)

    rec_class_search(root_node)
    return node_list

In [ ]:
#export
'''
Recursively searches an AST for method nodes

:param root_node: root node to begin search
:param key: key to search
:returns: list of method nodes
'''
def find_method_nodes(class_node, key):
    node_list = []
    def rec_method_search(node):
        if node.type == key["method"]:
            node_list.append(node)
        if node.type != key["class"]:
            for child in node.children:
                rec_method_search(child)
    
    for node in class_node.children:
        rec_method_search(node)
    return node_list

In [ ]:
#export
'''
Finds the fields/attributes for a class AST

:param class_node: class node to search
:param file_bytes: list of bytes
:param key: key to search
:returns: list of class fields
'''
def find_field_names(class_node, file_bytes, key):
    class_fields = []
    
    def rec_name_search(node):
        if node.type == key["field_name"]:
            word = []
            for i in range(node.start_byte, node.end_byte):
                word.append(file_bytes[i])
            class_fields.append(word)
        else:
            for child in node.children:
                rec_name_search(child)
    
    def rec_field_search(node):
        if node.type == key["field_dec"]:
            rec_name_search(node)
        if node.type != key["class"]:
            for child in node.children:
                rec_field_search(child)

    for node in class_node.children:
        rec_field_search(node)

    return class_fields

In [ ]:
#export
'''
Counts the number of occurences of a byte-pattern array in a sample of code

:param node:
:param pattern: pattern to look for
:param file_bytes: byte-array
:returns: number of occurrences
:raises IndexError: if file_bytes[i] not good
'''
def find_string_in_text(node, pattern, file_bytes):
    if len(node.children) > 0:
        count = 0
        for i in node.children:
            count += find_string_in_text(i, pattern, file_bytes)
        return count
    else:
        word = []
        for i in range(node.start_byte, node.end_byte):
            num_index_fails = 0
            try:
                word.append(file_bytes[i])
            except IndexError:
                num_index_fails += 1
        if(num_index_fails):
            print(f"INDEX ERROR ({num_index_fails} times)")
            print("Start byte:", node.start_byte, "End byte:", node.end_byte, "Word:", word)
        if word == pattern:
            return 1
        else:
            return 0

In [ ]:
#export
'''
Recursively searches an AST for method nodes

:param class_node:
:param field_names: 
:param file_bytes: byte array
:param key: key to search
:returns: number of total distinct calls
'''
def distinct_field_calls(class_node, field_names, file_bytes, key):
    total_distinct_calls = []
    
    def rec_method_search(node):
        if node.type == key["method"]:
            distinct_method_field_calls = 0
            for field in field_names:
                if find_string_in_text(node, field, file_bytes):
                    distinct_method_field_calls += 1
            total_distinct_calls.append(distinct_method_field_calls)
        if node.type != key["class"]:
            for child in node.children:
                rec_method_search(child)
    
    for node in class_node.children:
        rec_method_search(node)
        
    return len(total_distinct_calls)

In [ ]:
#export
'''
Parses the syntax tree of code to calculate the LCOM5 of its classes

:param tree: syntax tree
:param extension: file extension type
:param file_bytes: list of bytes
:param name: not used
:returns: LCOM5 list
'''
def calculate_lcom5(tree, extension, file_bytes, name):
    keyword_dict = lang_keywords()

    if extension not in keyword_dict:
        print(f"Tried to get LCOM5 of file with unsupported extension '.{extension}', 0 assigned to column.")
        return ["Undefined"]

    root_node = tree.root_node
    keywords = keyword_dict[extension]
    class_nodes = find_class_nodes(root_node, keywords)
    class_method_nodes = []
    class_field_names = []
    class_dfc = [] # Distinct field calls, as per the definition of LCOM5
    for node in enumerate(class_nodes):
        class_method_nodes.append(find_method_nodes(node[1], keywords))
        class_field_names.append(find_field_names(node[1], file_bytes, keywords))
        class_dfc.append(distinct_field_calls(node[1], class_field_names[node[0]], file_bytes, keywords))
    lcom5_list = []
    for j in range(len(class_nodes)):
        num_fields = len(class_field_names[j])
        num_meths = len(class_method_nodes[j])
        num_dac = class_dfc[j]
        numerator = num_dac - (num_meths*num_fields)
        denominator = num_fields - (num_meths*num_fields)
        if denominator == 0:
            lcom5_list.append("Undefined")
        else:
            lcom5_list.append(numerator/denominator)
    return lcom5_list

In [ ]:
#export
'''
Adds a column with the LCOM5 of each class of each file to a dataframe

:param df: input dataframe
:param col: not used
:returns: updated dataframe
'''
def add_lcom5(df, col):
    lang_builds = create_parser_builds()
    parser = Parser()
    class_lcom5 = []
    
    for i in range(len(df)):
        ext = df["name"][i].split('.')[-1]
        parser.set_language(lang_builds[ext])
        enc = df["encoding"][i]
        tree = parser.parse(bytes(df["contents"][i], df["encoding"][i]))
        class_lcom5.append(calculate_lcom5(tree, ext, bytes(df["contents"][i], df["encoding"][i]), df["name"][i]))
    df["class_lcom5"] = class_lcom5
    
    return df

In [ ]:
#export
'''
Takes in a list of lists and flattens it, returning a list of each entry

:param list_list: list of lists to flatten
:returns: flattened list
'''
def flatten_lol(list_list):
    flattened_list = []
    for sublist in list_list:
        for entry in sublist:
            flattened_list.append(entry)
    return flattened_list

In [ ]:
# export
'''
Computes statistical metrics about the entries in a dataframe column or list

:param col: dataframe column or list
:param conf: confidence level
:param sig_figs: significant figures for rounding
:param clean: only look at ints, floats, or complex in col
:param verbose_clean: displays number of non-numeric entries removed
'''
def display_numeric_col_stats(col, conf = 0.95, sig_figs = 4, clean=True, verbose_clean=False):
    previous_length = len(col)
    numeric_types = [int, float, complex]
    if clean: col = [x for x in col if type(x) in numeric_types]
    if verbose_clean: print(f"Cleaning removed {previous_length - len(col)} non-numeric entries")

    if len(col) < 1:
        print("Error, data must contain at least one valid entry to display statistics")
        return

    print("Min =", round(min(col), sig_figs))
    print("Max =", round(max(col), sig_figs))
    print("Average =", round(mean(col), sig_figs))
    print("Median =", round(median(col), sig_figs))
    print("Standard Deviation =", round(std(col), sig_figs))
    
    n = len(col)
    m = mean(col)
    std_err = sem(col)
    h = std_err * t.ppf((1 + conf) / 2, n - 1)

    start = m - h
    end = m + h
    print(f"{conf} of data points fall between {round(start, sig_figs)} and {round(end, sig_figs)}")

In [ ]:
#export
'''
Displays a histogram with a customized number of bins for the data in a specified dataframe column or list

:param col: df column or list to plot
:param col_name: name of col for labeling
:param num_bins: number of bins
:param clean: only look at ints, floats, or complex in col
:param verbose_clean: displays non-numeric entries removed
'''
def display_numeric_col_hist(col, col_name="Metric", num_bins=20, clean=True, verbose_clean=False):
    previous_length = len(col)
    numeric_types = [int, float, complex]
    if clean: col = [x for x in col if type(x) in numeric_types]
    if verbose_clean: print(f"Cleaning removed {previous_length - len(col)} non-numeric entries")

    if len(col) < 1:
        print("Error, data must contain at least one valid entry to display histogram")
        return    

    rng = max(col) - min(col)
    num = len(col)
    stnd_dev = std(col)

    plt.hist(col, num_bins, color="blue", alpha=0.5, edgecolor="black", linewidth=1.0)
    plt.title(col_name + " Histogram")
    plt.ylabel("Value  Range  Occurrences")
    plt.xlabel(col_name)
    plt.show()